In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K

Using TensorFlow backend.


In [2]:
dir = './sequence/*'

In [3]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [5]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 100

In [34]:
from keras.layers import Input, LSTM, RepeatVector, Lambda
from keras.models import Model
inputs = Input(shape=(None, 3))
encoded = LSTM(64)(inputs)


In [37]:
def repeat_vector(args):
        layer_to_repeat = args[0]
        sequence_layer = args[1]
        return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs])
decoded = LSTM(64, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3, activation='linear'))(decoded)

In [38]:
lstm_autoencoder = Model(inputs, decoded)

In [39]:
encoder = Model(inputs, encoded)

In [41]:
lstm_autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_15 (LSTM)                  (None, 64)           17408       input_12[0][0]                   
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, None, 64)     0           lstm_15[0][0]                    
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lstm_18 (LSTM)                  (None, None, 64)     33024       lambda_9[0][0]                   
__________

In [ ]:
from keras.optimizers import Adam
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=540, verbose=1)#, validation_data=val_generator(x_val))

In [21]:
# timesteps 맞춰주기 위해서 padding 추가
#all_data = sequence.pad_sequences(all_data, padding='post', maxlen = max_sequence_length)

In [34]:
#case 1
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(None, n_features), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1, activation="linear")))
model.add(LSTM(16, activation='relu', return_sequences=True))
model.add(LSTM(n_features, return_sequences=True))
#model.add(Dense(1))
#model.add(LSTM(n_features, return_sequences=True))
#model.add(LSTM(10, return_sequences=True))


In [52]:
#case 2
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(None, n_features), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(16, activation='relu', return_sequences=True))
model.add(LSTM(8, activation='relu', return_sequences=True))
#model.add(LSTM(n_features, return_sequences=True))
model.add(TimeDistributed(Dense(3, activation="linear")))
'''
0.0085, lr = 0.001
'''

'\n0.0085, lr = 0.001\n'

In [61]:
#case 3
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(None, n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(3, activation="linear")))
'''
0.0057 - 0.0113, lr = 0.001
'''

In [62]:
from keras.optimizers import Adam
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

In [63]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, None, 128)         67584     
_________________________________________________________________
lstm_55 (LSTM)               (None, None, 64)          49408     
_________________________________________________________________
lstm_56 (LSTM)               (None, None, 64)          33024     
_________________________________________________________________
lstm_57 (LSTM)               (None, None, 128)         98816     
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 3)           387       
Total params: 249,219
Trainable params: 249,219
Non-trainable params: 0
_________________________________________________________________


In [64]:
import random as rd
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
model.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=540, verbose=1)#, validation_data=val_generator(x_val))

Epoch 1/100
540/540 [==============================] - 18s 34ms/step - loss: 216238229.3050
Epoch 2/100
540/540 [==============================] - 11s 21ms/step - loss: 53.3566
Epoch 3/100
540/540 [==============================] - 11s 21ms/step - loss: 20.2062
Epoch 4/100
540/540 [==============================] - 11s 21ms/step - loss: 53.5959
Epoch 5/100
540/540 [==============================] - 11s 21ms/step - loss: 16.0597
Epoch 6/100
540/540 [==============================] - 11s 21ms/step - loss: 11.9949
Epoch 7/100
540/540 [==============================] - 11s 21ms/step - loss: 9.7418
Epoch 8/100
540/540 [==============================] - 11s 21ms/step - loss: 11.4632
Epoch 9/100
540/540 [==============================] - 12s 22ms/step - loss: 134.5107
Epoch 10/100
540/540 [==============================] - 11s 21ms/step - loss: 5.5216
Epoch 11/100
540/540 [==============================] - 11s 21ms/step - loss: 4.6954
Epoch 12/100
540/540 [==============================] - 12

In [65]:
#case 4
model4 = Sequential()
model4.add(LSTM(128, activation='relu', input_shape=(None, n_features), return_sequences=True)) #encoder
model4.add(TimeDistributed(Dense(3, activation="linear"))) #decoder
'''
2.0407e-04, lr = 0.0001
'''

In [67]:
model4.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
model4.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=540, verbose=1)

Epoch 1/100
540/540 [==============================] - 8s 14ms/step - loss: 56.5381
Epoch 2/100
540/540 [==============================] - 3s 6ms/step - loss: 0.7981
Epoch 3/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0983
Epoch 4/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0579
Epoch 5/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0419
Epoch 6/100
540/540 [==============================] - 4s 7ms/step - loss: 0.0314
Epoch 7/100
540/540 [==============================] - 4s 7ms/step - loss: 0.0234
Epoch 8/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0143
Epoch 9/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0078
Epoch 10/100
540/540 [==============================] - 4s 7ms/step - loss: 0.0057
Epoch 11/100
540/540 [==============================] - 3s 6ms/step - loss: 0.0045
Epoch 12/100
540/540 [==============================] - 4s 7ms/step - loss: 0.0040
Epoch 13/10

In [72]:
#case 5
model5 = Sequential()
model5.add(LSTM(128, activation='relu', input_shape=(None, n_features), return_sequences=True)) #encoder
model5.add(LSTM(128, activation='relu', return_sequences=True))
model5.add(TimeDistributed(Dense(3, activation="linear"))) #decoder
'''
epochs = 100, 0.0020, lr = 0.0001
epochs = 500, 8.0520e-04, loss, lr=0.0001
'''

'\n0.0020, lr = 0.0001\n'

In [ ]:
model5.

In [73]:
model5.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
model5.fit_generator(train_generator(x_train), epochs=150, steps_per_epoch=540, verbose=1)

Epoch 1/500
540/540 [==============================] - 12s 21ms/step - loss: 285441.2934
Epoch 2/500
540/540 [==============================] - 6s 12ms/step - loss: 5.6163
Epoch 3/500
540/540 [==============================] - 6s 12ms/step - loss: 3.1137
Epoch 4/500
540/540 [==============================] - 6s 12ms/step - loss: 2735407.8253
Epoch 5/500
540/540 [==============================] - 6s 12ms/step - loss: 17.6709
Epoch 6/500
540/540 [==============================] - 6s 12ms/step - loss: 12.2263
Epoch 7/500
540/540 [==============================] - 6s 12ms/step - loss: 11.4329
Epoch 8/500
540/540 [==============================] - 6s 12ms/step - loss: 10.5815
Epoch 9/500
540/540 [==============================] - 6s 12ms/step - loss: 8.8390
Epoch 10/500
540/540 [==============================] - 6s 12ms/step - loss: 9.1986
Epoch 11/500
540/540 [==============================] - 6s 12ms/step - loss: 5.1294
Epoch 12/500
540/540 [==============================] - 6s 12ms/step 

KeyboardInterrupt: 

In [70]:
#case 6
model6 = Sequential()
model6.add(LSTM(64, activation='relu', input_shape=(None, n_features), return_sequences=True)) #encoder
model6.add(LSTM(64, activation='relu', return_sequences=True))
model6.add(TimeDistributed(Dense(3, activation="linear"))) #decoder
'''
ep=100, 6.8112e-04, lr=0.0001
'''
model6.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
model6.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=540, verbose=1)

Epoch 1/100
540/540 [==============================] - 10s 18ms/step - loss: 5.0510
Epoch 2/100
540/540 [==============================] - 6s 11ms/step - loss: 0.1207
Epoch 3/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0408
Epoch 4/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0165
Epoch 5/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0113
Epoch 6/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0085
Epoch 7/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0089
Epoch 8/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0073
Epoch 9/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0067
Epoch 10/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0066
Epoch 11/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0051
Epoch 12/100
540/540 [==============================] - 6s 11ms/step - loss: 0.0057


In [71]:
#case 7
model7 = Sequential()
model7.add(LSTM(64, activation='relu', input_shape=(None, n_features), return_sequences=True)) #encoder
model7.add(LSTM(64, activation='relu', return_sequences=True))#decoder
model7.add(TimeDistributed(Dense(3, activation='relu'))) 
'''
6.8112e-04, lr=0.0001
'''
model7.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
model7.fit_generator(train_generator(x_train), epochs=1000, steps_per_epoch=540, verbose=1)

Epoch 1/1000
540/540 [==============================] - 11s 20ms/step - loss: 6.5372
Epoch 2/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.1494
Epoch 3/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0474
Epoch 4/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0255
Epoch 5/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0142
Epoch 6/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0106
Epoch 7/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0085
Epoch 8/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0078
Epoch 9/1000
540/540 [==============================] - 6s 10ms/step - loss: 0.0080
Epoch 10/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0072
Epoch 11/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0053
Epoch 12/1000
540/540 [==============================] - 6s 10ms/step - l

540/540 [==============================] - 6s 11ms/step - loss: 7.6960e-04
Epoch 97/1000
540/540 [==============================] - 6s 11ms/step - loss: 4.1963e-04
Epoch 98/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0012
Epoch 99/1000
540/540 [==============================] - 6s 11ms/step - loss: 3.7665e-04
Epoch 100/1000
540/540 [==============================] - 6s 11ms/step - loss: 6.2618e-04
Epoch 101/1000
540/540 [==============================] - 6s 11ms/step - loss: 3.8553e-04
Epoch 102/1000
540/540 [==============================] - 6s 11ms/step - loss: 0.0013
Epoch 103/1000
540/540 [==============================] - 6s 10ms/step - loss: 4.4139e-04
Epoch 104/1000
540/540 [==============================] - 6s 10ms/step - loss: 7.9998e-04
Epoch 105/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.7952e-04
Epoch 106/1000
540/540 [==============================] - 6s 10ms/step - loss: 7.2096e-04
Epoch 107/1000
540/540 [============

540/540 [==============================] - 6s 10ms/step - loss: 4.8088e-04
Epoch 279/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.9103e-04
Epoch 280/1000
540/540 [==============================] - 6s 10ms/step - loss: 3.8522e-04
Epoch 281/1000
540/540 [==============================] - 6s 10ms/step - loss: 4.4214e-04
Epoch 282/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.6142e-04
Epoch 283/1000
540/540 [==============================] - 6s 10ms/step - loss: 5.6092e-04
Epoch 284/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.1917e-04
Epoch 285/1000
540/540 [==============================] - 6s 10ms/step - loss: 4.4640e-04
Epoch 286/1000
540/540 [==============================] - 6s 11ms/step - loss: 3.4037e-04
Epoch 287/1000
540/540 [==============================] - 6s 11ms/step - loss: 3.2743e-04
Epoch 288/1000
540/540 [==============================] - 6s 11ms/step - loss: 1.7525e-04
Epoch 289/1000
540/540 [=

540/540 [==============================] - 6s 11ms/step - loss: 1.9574e-04
Epoch 461/1000
540/540 [==============================] - 6s 11ms/step - loss: 2.9558e-04
Epoch 462/1000
540/540 [==============================] - 6s 11ms/step - loss: 2.1447e-04
Epoch 463/1000
540/540 [==============================] - 6s 11ms/step - loss: 1.5616e-04
Epoch 464/1000
540/540 [==============================] - 6s 11ms/step - loss: 4.1528e-04
Epoch 465/1000
540/540 [==============================] - 6s 11ms/step - loss: 2.9077e-04
Epoch 466/1000
540/540 [==============================] - 6s 11ms/step - loss: 1.6621e-04
Epoch 467/1000
540/540 [==============================] - 6s 11ms/step - loss: 1.7057e-04
Epoch 468/1000
540/540 [==============================] - 6s 11ms/step - loss: 6.4942e-04
Epoch 469/1000
540/540 [==============================] - 6s 11ms/step - loss: 1.2861e-04
Epoch 470/1000
540/540 [==============================] - 6s 10ms/step - loss: 3.3817e-04
Epoch 471/1000
540/540 [=

540/540 [==============================] - 6s 10ms/step - loss: 4.2518e-04
Epoch 643/1000
540/540 [==============================] - 6s 10ms/step - loss: 3.0175e-04
Epoch 644/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.0675e-04
Epoch 645/1000
540/540 [==============================] - 6s 10ms/step - loss: 1.0857e-04
Epoch 646/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.7958e-04
Epoch 647/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.0516e-04
Epoch 648/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.6660e-04
Epoch 649/1000
540/540 [==============================] - 6s 10ms/step - loss: 1.7713e-04
Epoch 650/1000
540/540 [==============================] - 6s 10ms/step - loss: 3.0864e-04
Epoch 651/1000
540/540 [==============================] - 6s 10ms/step - loss: 2.1886e-04
Epoch 652/1000
540/540 [==============================] - 6s 10ms/step - loss: 1.6767e-04
Epoch 653/1000
540/540 [=

KeyboardInterrupt: 

In [43]:
#case 7
model8 = Sequential()
model8.add(LSTM(64, activation='relu', input_shape=(None, n_features), return_sequences=True)) #encoder
#model8.add(RepeatVector(layer))
model8.add(LSTM(64, activation='relu', return_sequences=True))#decoder
model8.add(TimeDistributed(Dense(3, activation='relu'))) 
model8.summary()
model8.layers[0].get_output_shape_at(0)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, None, 64)          17408     
_________________________________________________________________
lstm_40 (LSTM)               (None, None, 64)          33024     
_________________________________________________________________
time_distributed_15 (TimeDis (None, None, 3)           195       
Total params: 50,627
Trainable params: 50,627
Non-trainable params: 0
_________________________________________________________________


TypeError: get_output_shape_at() missing 1 required positional argument: 'node_index'

In [25]:
model8.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 64)                17408     
Total params: 17,408
Trainable params: 17,408
Non-trainable params: 0
_________________________________________________________________


In [15]:
from keras.optimizers import Adam
'''
6.8112e-04, lr=0.0001
'''
model8.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
model8.fit_generator(train_generator(x_train), epochs=1000, steps_per_epoch=540, verbose=1)

RuntimeError: You must compile your model before using it.

In [12]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0